In [4]:
!pip install speechrecognition textblob scikit-learn pandas joblib
!pip install pyttsx3  # For text-to-speech conversion (optional for better interaction)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 58.0 MB/s eta 0:00:00


In [9]:
import os
import speech_recognition as sr
from textblob import TextBlob
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
import pandas as pd
import joblib
from google.colab import files

# -------- TRAIN MODEL (with evaluation metrics) --------
def train_model():
    print("🔧 Training model...")
    data = {
        'SpeechText': [
            "I am happy with the service",
            "I'm very angry and frustrated",
            "Everything is fine",
            "I’m disappointed with this",
            "It’s really good",
            "Worst experience ever"
        ],
        'Churned': [0, 1, 0, 1, 0, 1]
    }

    df = pd.DataFrame(data)

    def extract_features(text):
        blob = TextBlob(text)
        sentiment = blob.sentiment.polarity
        emotion_words = ['angry', 'frustrated', 'sad', 'happy', 'disappointed', 'satisfied']
        count = sum(word in text.lower() for word in emotion_words)
        return [sentiment, count]

    df[['Sentiment', 'EmotionCount']] = df['SpeechText'].apply(lambda x: pd.Series(extract_features(x)))

    X = df[['Sentiment', 'EmotionCount']]
    y = df['Churned']

    # Train-test split for evaluation
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)

    # Predict on test set
    y_pred = model.predict(X_test)

    # Evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, zero_division=0)
    recall = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    cm = confusion_matrix(y_test, y_pred)

    print("\n📈 Evaluation Metrics:")
    print(f"✅ Accuracy: {accuracy:.2f}")
    print(f"✅ Precision: {precision:.2f}")
    print(f"✅ Recall: {recall:.2f}")
    print(f"✅ F1 Score: {f1:.2f}")
    print("✅ Confusion Matrix:\n", cm)

    # Save model
    joblib.dump(model, "churn_model.pkl")
    print("\n💾 Model saved as churn_model.pkl")
    return model

# -------- UPLOAD AUDIO FILE --------
def upload_audio():
    print("⏬ Upload an audio file (in .wav format) for speech-to-text conversion.")
    uploaded = files.upload()
    for audio_file in uploaded.keys():
        return audio_file

# -------- CONVERT AUDIO TO TEXT --------
def recognize_speech_from_file(audio_file):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file) as source:
        print("🎧 Converting audio to text...")
        audio = recognizer.record(source)
    try:
        text = recognizer.recognize_google(audio)
        print("📝 Transcribed Text:", text)
        return text
    except:
        print("❌ Could not understand the audio.")
        return ""

# -------- EXTRACT EMOTION FEATURES --------
def extract_emotion_features(text):
    blob = TextBlob(text)
    sentiment = blob.sentiment.polarity
    emotion_words = ['angry', 'frustrated', 'sad', 'happy', 'disappointed', 'satisfied']
    count = sum(word in text.lower() for word in emotion_words)
    return [sentiment, count]

# -------- MAIN --------
def main():
    model = train_model()  # Always retrains and prints metrics

    audio_file = upload_audio()  # Upload your audio file
    if not audio_file:
        print("❌ No audio file uploaded.")
        return

    text = recognize_speech_from_file(audio_file)  # Convert audio to text
    if not text:
        return

    features = extract_emotion_features(text)
    print("📊 Features: Sentiment =", features[0], ", Emotion Count =", features[1])

    prediction = model.predict([features])[0]
    print("🔮 Prediction:", "CHURN" if prediction == 1 else "STAY")

if __name__ == "__main__":
    main()


🔧 Training model...

📈 Evaluation Metrics:
✅ Accuracy: 1.00
✅ Precision: 1.00
✅ Recall: 1.00
✅ F1 Score: 1.00
✅ Confusion Matrix:
 [[1 0]
 [0 1]]

💾 Model saved as churn_model.pkl
⏬ Upload an audio file (in .wav format) for speech-to-text conversion.


Saving happy .wav.wav to happy .wav.wav
🎧 Converting audio to text...
📝 Transcribed Text: I'm very happy with your service
📊 Features: Sentiment = 1.0 , Emotion Count = 1
🔮 Prediction: STAY


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
